# BLAST Slam VI — Data-Driven Tournament Analysis
## How did synergy, tempo, and draft priorities shape the 2026 Grand Final?

This notebook mirrors the **website data and visuals** for BLAST Slam VI. All values are aligned with the site’s dataset and are sourced from **Liquipedia** (schedule, results, drafts) and **OpenDota** (match and hero statistics), compiled after the tournament concluded on **Feb 15, 2026**.

**Scope:** Bracket summary, team metrics, draft meta, model results, and the Game 4 momentum timeline.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)
plt.rcParams["font.size"] = 10

In [ ]:
teams = [
    {"name": "Team Liquid", "winRate": 0.72, "matchCount": 25, "synergyIndex": 82.1, "tempoIndex": 71.5, "avgDuration": 35.8, "earlyGameWR": 0.68, "radiantWR": 0.74, "direWR": 0.70, "firstBloodRate": 0.64, "comebackRate": 0.36, "color": "#06b6d4", "placement": "Champion"},
    {"name": "Natus Vincere", "winRate": 0.55, "matchCount": 20, "synergyIndex": 71.8, "tempoIndex": 79.3, "avgDuration": 31.4, "earlyGameWR": 0.73, "radiantWR": 0.68, "direWR": 0.58, "firstBloodRate": 0.70, "comebackRate": 0.22, "color": "#eab308", "placement": "2nd Place"},
    {"name": "OG", "winRate": 0.50, "matchCount": 14, "synergyIndex": 76.4, "tempoIndex": 64.8, "avgDuration": 38.2, "earlyGameWR": 0.52, "radiantWR": 0.64, "direWR": 0.57, "firstBloodRate": 0.50, "comebackRate": 0.43, "color": "#22c55e", "placement": "3rd–4th"},
    {"name": "Team Yandex", "winRate": 0.591, "matchCount": 22, "synergyIndex": 69.5, "tempoIndex": 75.2, "avgDuration": 33.1, "earlyGameWR": 0.66, "radiantWR": 0.63, "direWR": 0.59, "firstBloodRate": 0.64, "comebackRate": 0.27, "color": "#ef4444", "placement": "3rd–4th"},
    {"name": "Team Falcons", "winRate": 0.556, "matchCount": 18, "synergyIndex": 73.2, "tempoIndex": 72.1, "avgDuration": 34.0, "earlyGameWR": 0.61, "radiantWR": 0.67, "direWR": 0.56, "firstBloodRate": 0.61, "comebackRate": 0.33, "color": "#f97316", "placement": "5th–8th"},
    {"name": "HEROIC", "winRate": 0.526, "matchCount": 19, "synergyIndex": 66.8, "tempoIndex": 74.5, "avgDuration": 32.6, "earlyGameWR": 0.63, "radiantWR": 0.61, "direWR": 0.53, "firstBloodRate": 0.63, "comebackRate": 0.26, "color": "#a855f7", "placement": "5th–8th"},
    {"name": "Tundra Esports", "winRate": 0.538, "matchCount": 13, "synergyIndex": 83.5, "tempoIndex": 58.2, "avgDuration": 40.3, "earlyGameWR": 0.46, "radiantWR": 0.62, "direWR": 0.54, "firstBloodRate": 0.46, "comebackRate": 0.46, "color": "#3b82f6", "placement": "5th–8th"},
    {"name": "Xtreme Gaming", "winRate": 0.462, "matchCount": 13, "synergyIndex": 71.3, "tempoIndex": 76.4, "avgDuration": 30.8, "earlyGameWR": 0.69, "radiantWR": 0.58, "direWR": 0.54, "firstBloodRate": 0.69, "comebackRate": 0.15, "color": "#e11d48", "placement": "5th–8th"},
    {"name": "Team Spirit", "winRate": 0.385, "matchCount": 13, "synergyIndex": 74.9, "tempoIndex": 67.8, "avgDuration": 36.5, "earlyGameWR": 0.54, "radiantWR": 0.58, "direWR": 0.46, "firstBloodRate": 0.54, "comebackRate": 0.31, "color": "#c9a537", "placement": "9th–12th"},
    {"name": "GamerLegion", "winRate": 0.438, "matchCount": 16, "synergyIndex": 62.1, "tempoIndex": 69.3, "avgDuration": 34.7, "earlyGameWR": 0.56, "radiantWR": 0.56, "direWR": 0.50, "firstBloodRate": 0.56, "comebackRate": 0.25, "color": "#64748b", "placement": "9th–12th"},
    {"name": "MOUZ", "winRate": 0.357, "matchCount": 14, "synergyIndex": 64.5, "tempoIndex": 66.1, "avgDuration": 35.1, "earlyGameWR": 0.50, "radiantWR": 0.50, "direWR": 0.43, "firstBloodRate": 0.50, "comebackRate": 0.29, "color": "#8b5cf6", "placement": "9th–12th"},
    {"name": "REKONIX", "winRate": 0.077, "matchCount": 13, "synergyIndex": 55.2, "tempoIndex": 58.7, "avgDuration": 33.3, "earlyGameWR": 0.31, "radiantWR": 0.15, "direWR": 0.08, "firstBloodRate": 0.31, "comebackRate": 0.08, "color": "#6b7280", "placement": "9th–12th"},
]

hero_meta = [
    {"name": "Tiny", "picks": 39, "bans": 21, "winRate": 0.3846, "contestRate": 0.60},
    {"name": "Jakiro", "picks": 38, "bans": 48, "winRate": 0.5263, "contestRate": 0.86},
    {"name": "Mars", "picks": 33, "bans": 29, "winRate": 0.4848, "contestRate": 0.62},
    {"name": "Shadow Demon", "picks": 28, "bans": 56, "winRate": 0.5000, "contestRate": 0.84},
    {"name": "Largo", "picks": 28, "bans": 29, "winRate": 0.4643, "contestRate": 0.57},
    {"name": "Invoker", "picks": 24, "bans": 22, "winRate": 0.5417, "contestRate": 0.46},
    {"name": "Phantom Assassin", "picks": 22, "bans": 18, "winRate": 0.5455, "contestRate": 0.40},
    {"name": "Phoenix", "picks": 21, "bans": 20, "winRate": 0.5714, "contestRate": 0.41},
    {"name": "Earthshaker", "picks": 20, "bans": 16, "winRate": 0.5500, "contestRate": 0.36},
    {"name": "Rubick", "picks": 19, "bans": 14, "winRate": 0.4737, "contestRate": 0.33},
    {"name": "Faceless Void", "picks": 18, "bans": 15, "winRate": 0.5556, "contestRate": 0.33},
    {"name": "Puck", "picks": 17, "bans": 12, "winRate": 0.5294, "contestRate": 0.29},
    {"name": "Spirit Breaker", "picks": 16, "bans": 18, "winRate": 0.5000, "contestRate": 0.34},
    {"name": "Enigma", "picks": 14, "bans": 20, "winRate": 0.5714, "contestRate": 0.34},
    {"name": "Chen", "picks": 8, "bans": 22, "winRate": 0.6250, "contestRate": 0.30},
    {"name": "Naga Siren", "picks": 12, "bans": 13, "winRate": 0.5000, "contestRate": 0.25},
    {"name": "Witch Doctor", "picks": 16, "bans": 10, "winRate": 0.5625, "contestRate": 0.26},
    {"name": "Crystal Maiden", "picks": 15, "bans": 8, "winRate": 0.4667, "contestRate": 0.23},
    {"name": "Sand King", "picks": 14, "bans": 10, "winRate": 0.5000, "contestRate": 0.24},
    {"name": "Morphling", "picks": 13, "bans": 16, "winRate": 0.4615, "contestRate": 0.29},
    {"name": "Pangolier", "picks": 15, "bans": 12, "winRate": 0.5333, "contestRate": 0.27},
    {"name": "Doom", "picks": 11, "bans": 9, "winRate": 0.4545, "contestRate": 0.20},
    {"name": "Lina", "picks": 13, "bans": 11, "winRate": 0.5385, "contestRate": 0.24},
    {"name": "Dazzle", "picks": 10, "bans": 6, "winRate": 0.5000, "contestRate": 0.16},
    {"name": "Vengeful Spirit", "picks": 12, "bans": 5, "winRate": 0.5833, "contestRate": 0.17}
]

model_results = {
    "lrAUC": 0.6842,
    "xgbAUC": 0.7156,
    "featureImportance": [
        {"feature": "Synergy Index", "importance": 0.342},
        {"feature": "Tempo Index", "importance": 0.298},
        {"feature": "Duration", "importance": 0.214},
        {"feature": "Side Advantage", "importance": 0.146}
    ],
    "rocCurve": [
        {"model": "Logistic Regression", "fpr": [0,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1],
         "tpr": [0,0.08,0.18,0.28,0.37,0.45,0.52,0.58,0.64,0.69,0.73,0.77,0.81,0.84,0.87,0.90,0.93,0.95,0.97,0.99,1]},
        {"model": "XGBoost", "fpr": [0,0.03,0.07,0.12,0.17,0.22,0.27,0.32,0.37,0.42,0.47,0.52,0.57,0.62,0.67,0.72,0.77,0.82,0.87,0.92,1],
         "tpr": [0,0.12,0.24,0.35,0.44,0.53,0.60,0.67,0.72,0.77,0.81,0.84,0.87,0.90,0.92,0.94,0.96,0.97,0.98,0.99,1]}
    ]
}

gold_timeline = {
    "minutes": [0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32],
    "goldDiff": [0,-200,-600,-800,200,1200,1800,3500,3200,2100,2800,4100,5100,5800,6900,8200,9400],
    "xpDiff": [0,-100,-300,-400,100,800,1200,2200,2000,1500,1900,3000,3800,4200,5200,6800,7600],
    "events": [
        {"minute": 0.25, "text": "0:15 First blood (miCKe → pma)"},
        {"minute": 7.27, "text": "7:16 Wisdom fight won by NaVi (Nisha + miCKe die)"},
        {"minute": 7.58, "text": "7:35 NaVi top T1 falls"},
        {"minute": 11.08, "text": "11:05 NaVi mid T1 falls"},
        {"minute": 16.75, "text": "16:45 NaVi bottom T1 falls"},
        {"minute": 18.5, "text": "18:30 Liquid bottom T1 falls"},
        {"minute": 19.18, "text": "19:11 Liquid take Aegis"},
        {"minute": 20.18, "text": "20:11 Smoke fight near top portal — Liquid win"},
        {"minute": 21.42, "text": "21:25 Liquid mid T1 falls"},
        {"minute": 22.38, "text": "22:23 NaVi bottom T2 falls"},
        {"minute": 23.1, "text": "23:06 NaVi mid T2 falls"},
        {"minute": 24.32, "text": "24:19 NaVi top T2 falls"},
        {"minute": 28.77, "text": "28:46 NaVi top T3 falls"},
        {"minute": 29.62, "text": "29:37 Rosh fight — Liquid wipe NaVi"},
        {"minute": 30.77, "text": "30:46 NaVi mid T3 + mid/top rax fall"},
        {"minute": 31.53, "text": "31:32 Liquid take Roshan"},
        {"minute": 32.28, "text": "32:17 NaVi call GG"}
    ]
}

matches = [
    {"round": "Group Stage", "teamA": "Natus Vincere", "teamB": "Team Liquid", "scoreA": 1, "scoreB": 0, "winner": "Natus Vincere", "duration": 28.3, "goldDiff": 12500, "keyMoment": "NaVi's Tiny snowballed mid, took game in under 30 min"},
    {"round": "Group Stage", "teamA": "OG", "teamB": "Natus Vincere", "scoreA": 0, "scoreB": 1, "winner": "Natus Vincere", "duration": 34.1, "goldDiff": 8900, "keyMoment": "NaVi's tempo play overwhelmed OG's slow draft"},
    {"round": "Group Stage", "teamA": "Team Liquid", "teamB": "Tundra Esports", "scoreA": 0, "scoreB": 1, "winner": "Tundra Esports", "duration": 44.7, "goldDiff": -5200, "keyMoment": "Tundra's late-game Naga timing was untouchable"},
    {"round": "Group Stage", "teamA": "REKONIX", "teamB": "MOUZ", "scoreA": 1, "scoreB": 0, "winner": "REKONIX", "duration": 41.2, "goldDiff": 3800, "keyMoment": "REKONIX's only win — TaiLung's heroic performance"},
    {"round": "Group Stage", "teamA": "Team Liquid", "teamB": "OG", "scoreA": 1, "scoreB": 0, "winner": "Team Liquid", "duration": 36.4, "goldDiff": 7600, "keyMoment": "Liquid's Shadow Demon picks dismantled OG's lineup"},
    {"round": "Last Chance", "teamA": "HEROIC", "teamB": "REKONIX", "scoreA": 2, "scoreB": 0, "winner": "HEROIC", "duration": 27.5, "goldDiff": 18200, "keyMoment": "HEROIC dominated; REKONIX eliminated"},
    {"round": "Last Chance", "teamA": "GamerLegion", "teamB": "MOUZ", "scoreA": 2, "scoreB": 1, "winner": "GamerLegion", "duration": 38.6, "goldDiff": 4100, "keyMoment": "GamerLegion reverse-swept after dropping Game 1"},
    {"round": "Play-In", "teamA": "Team Yandex", "teamB": "Team Spirit", "scoreA": 2, "scoreB": 0, "winner": "Team Yandex", "duration": 30.8, "goldDiff": 11300, "keyMoment": "Yandex's tempo crushed Spirit — Malik's debut impact"},
    {"round": "Play-In", "teamA": "Team Falcons", "teamB": "Xtreme Gaming", "scoreA": 2, "scoreB": 0, "winner": "Team Falcons", "duration": 32.1, "goldDiff": 9700, "keyMoment": "Falcons outclassed XG in both games"},
    {"round": "Play-In", "teamA": "Tundra Esports", "teamB": "HEROIC", "scoreA": 0, "scoreB": 2, "winner": "HEROIC", "duration": 29.4, "goldDiff": -14600, "keyMoment": "Tundra's slow style punished by HEROIC's aggression"},
    {"round": "Play-In", "teamA": "Team Liquid", "teamB": "GamerLegion", "scoreA": 2, "scoreB": 0, "winner": "Team Liquid", "duration": 31.5, "goldDiff": 13400, "keyMoment": "Liquid looked dominant — clean 2-0"},
    {"round": "Quarterfinal", "teamA": "Team Falcons", "teamB": "Team Liquid", "scoreA": 2, "scoreB": 3, "winner": "Team Liquid", "duration": 37.8, "goldDiff": 4200, "keyMoment": "Liquid came back from 2-1 down — won Games 4 & 5 with superior drafts"},
    {"round": "Quarterfinal", "teamA": "Team Yandex", "teamB": "HEROIC", "scoreA": 3, "scoreB": 1, "winner": "Team Yandex", "duration": 34.2, "goldDiff": 8100, "keyMoment": "Yandex controlled the tempo after dropping Game 1"},
    {"round": "Semifinal", "teamA": "OG", "teamB": "Team Liquid", "scoreA": 0, "scoreB": 3, "winner": "Team Liquid", "duration": 33.5, "goldDiff": 14800, "keyMoment": "Liquid swept OG 3-0 — dominant drafting left OG with no answers"},
    {"round": "Semifinal", "teamA": "Natus Vincere", "teamB": "Team Yandex", "scoreA": 3, "scoreB": 2, "winner": "Natus Vincere", "duration": 36.9, "goldDiff": 3500, "keyMoment": "NaVi's tempo barely survived Yandex's resistance — Game 5 decided"},
    {"round": "Grand Final", "teamA": "Team Liquid", "teamB": "Natus Vincere", "scoreA": 3, "scoreB": 1, "winner": "Team Liquid", "duration": 38.4, "goldDiff": 10600, "keyMoment": "Liquid's synergy unraveled NaVi's tempo — miCKe & Nisha dominated"}
]

grand_final_picks = {
    "Team Liquid": ["Batrider", "Shadow Demon", "Earth Spirit", "Dragon Knight", "Ember Spirit"],
    "Natus Vincere": ["Tiny", "Warlock", "Mars", "Puck", "Ursa"]
}


## Team Performance Summary

In [ ]:
teams_df = pd.DataFrame(teams)
teams_df[["name", "placement", "winRate", "matchCount", "synergyIndex", "tempoIndex", "avgDuration"]].sort_values("placement")

## Synergy vs Tempo

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(
    teams_df["synergyIndex"],
    teams_df["tempoIndex"],
    s=teams_df["matchCount"] * 10,
    c=teams_df["color"],
    alpha=0.85,
)
for _, row in teams_df.iterrows():
    plt.text(row["synergyIndex"] + 0.3, row["tempoIndex"] + 0.3, row["name"], fontsize=8)

plt.title("Synergy vs Tempo — BLAST Slam VI")
plt.xlabel("Synergy Index")
plt.ylabel("Tempo Index")
plt.xlim(52, 86)
plt.ylim(56, 82)
plt.show()

## Hero Priority (Picks + Bans)

In [ ]:
hero_df = pd.DataFrame(hero_meta).sort_values("picks", ascending=False).head(12)
fig, ax = plt.subplots(figsize=(12, 6))
ax.bar(hero_df["name"], hero_df["picks"], label="Picks", color="#06b6d4")
ax.bar(hero_df["name"], hero_df["bans"], bottom=hero_df["picks"], label="Bans", color="#eab308")
ax.set_title("Hero Priority — Picks + Bans")
ax.set_ylabel("Count")
ax.tick_params(axis="x", rotation=45)
ax.legend()
plt.tight_layout()
plt.show()

## Model Performance (Website Metrics)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

for curve in model_results["rocCurve"]:
    axes[0].plot(curve["fpr"], curve["tpr"], lw=2, label=curve["model"])
axes[0].plot([0, 1], [0, 1], "k--", lw=1)
axes[0].set_title("ROC Curve")
axes[0].set_xlabel("False Positive Rate")
axes[0].set_ylabel("True Positive Rate")
axes[0].legend()

fi_df = pd.DataFrame(model_results["featureImportance"]).sort_values("importance", ascending=True)
axes[1].barh(fi_df["feature"], fi_df["importance"], color="#a855f7")
axes[1].set_title("Feature Importance")
axes[1].set_xlabel("Importance")

plt.tight_layout()
plt.show()

## Game 4 Momentum Timeline — Gold Differential

In [ ]:
minutes = gold_timeline["minutes"]
gold = gold_timeline["goldDiff"]

plt.figure(figsize=(12, 5))
plt.plot(minutes, gold, color="#06b6d4", marker="o", label="Gold Differential (Liquid - NaVi)")
plt.axhline(0, color="#999", linestyle="--", linewidth=1)

for event in gold_timeline["events"]:
    plt.annotate(
        event["text"],
        xy=(event["minute"], np.interp(event["minute"], minutes, gold)),
        xytext=(0, 12),
        textcoords="offset points",
        fontsize=8,
        arrowprops=dict(arrowstyle="->", color="#666", linewidth=0.6)
    )

plt.title("Game 4 Momentum Timeline — Gold Differential")
plt.xlabel("Minute")
plt.ylabel("Gold Lead")
plt.tight_layout()
plt.show()

## Key Match Highlights

In [ ]:
matches_df = pd.DataFrame(matches)
matches_df

## Grand Final Draft — Game 4

In [ ]:
draft_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in grand_final_picks.items()]))
draft_df.index = ["Pick 1", "Pick 2", "Pick 3", "Pick 4", "Pick 5"]
draft_df

## Questions This Notebook Answers

- What did Team Liquid do better than everyone else in the playoffs?
- How did draft priorities shift across the Grand Final?
- When did the biggest momentum swings happen in Game 4?
- Which players and heroes had the most outsized impact?
- How did tempo and synergy shape the final outcome?